In [1]:
import os
os.makedirs('output', exist_ok=True)

import sys
sys.path.append('../')

from utils import load_walksheds, overlay_wks
import pandas as pd
import numpy as np
import geopandas as gpd
import os

* wksp5, wksp75  are the Metro station walksheds with overlapping boundaries at 0.5 and 0.75 miles respectively
* all_boundaries All boundaries for census blocks in MD, DC and VA
* sld is the dataframe for the EPA smartlocation dataset

In [2]:
from IPython.display import Audio

def play_sound():
    display(Audio(url='https://proxy.notificationsounds.com/free-jingles-and-logos/light-hearted-message-tone/download/file-sounds-1351-light-hearted.mp3', autoplay=True))


# play_sound()

In [3]:
wksp5, wksp75 = load_walksheds()
all_boundaries= gpd.read_file('../../Data/Merge Boundaries/MD_DC_VA/Block MD_DC_VA.shp')
sld = pd.read_csv("../../Data/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")

* transauto is a subset of the sld dataframe with the accessibility by transit and auto variable as well as the geo identifiers <br/>

* used zfill to ensure that the values in the statefp, countyfp, and tractce, must be 2 digit, 3 diigits and 6 digits respectively, by adding zeros in front of the values

* join_sld_Boundaries is a merge of the transauto dataframe and the all_boundaries dataframe on their geo-identifiers. Keep boundaries where there are no jobs

* Change the CRS of the boundaries to match the walksheds

In [4]:
transauto = sld[['GEOID20', 'COUNTYFP', 'STATEFP', 'TRACTCE', 'D5BR', 'D5AR', 'Shape_Area']].copy()
transauto["STATEFP"] = transauto["STATEFP"].astype('str').str.zfill(2)
transauto["COUNTYFP"] = transauto["COUNTYFP"].astype('str').str.zfill(3)
transauto["TRACTCE"] = transauto["TRACTCE"].astype('str').str.zfill(6)
transauto['GEOID20'] = transauto['STATEFP'] + transauto['COUNTYFP'] + transauto['TRACTCE']

In [5]:
all_boundaries['GEOID200'] = all_boundaries['STATEFP20'] + all_boundaries['COUNTYFP20'] + all_boundaries['TRACTCE20']
Join_sld_Boundaries=pd.merge(all_boundaries, transauto, left_on="GEOID200", right_on = "GEOID20", how='outer')

import warnings # comment this out to show Janna warning 
warnings.filterwarnings('ignore')
Join_sld_Boundaries = Join_sld_Boundaries.to_crs('EPSG:4326')
Join_sld_Boundaries['fullarea'] = Join_sld_Boundaries.area

* Create a column of the full area of the jobs* Intersect the jobs with the walksheds
* Create a partial area column to get the block area that is within a walkshed
* Fix the names of the stations by removing extra numbers/characters at the end

In [6]:
intersect_stations_sld_p5, intersect_stations_sld_p75 = overlay_wks(Join_sld_Boundaries)

In [7]:
intersect_stations_sld_p5['D5BR'].loc[intersect_stations_sld_p5['D5BR'] < 0] = np.nan
intersect_stations_sld_p5['D5AR'].loc[intersect_stations_sld_p5['D5BR'] < 0] = np.nan

intersect_stations_sld_p75['D5BR'].loc[intersect_stations_sld_p75['D5BR'] < 0] = np.nan
intersect_stations_sld_p75['D5AR'].loc[intersect_stations_sld_p75['D5BR'] < 0] = np.nan

* Sum of Jobs accessible in 45mins by Auto

#### Measure of jobs or working age population within a 45 minute commute by transit or by auto

In [8]:
auto_trans_p5 = intersect_stations_sld_p5.groupby(['Name_1'])[['D5AR', 'D5BR']].apply(sum, axis=0)
auto_trans_p75 = intersect_stations_sld_p75.groupby(['Name_1'])[['D5AR', 'D5BR']].apply(sum, axis=0)

In [9]:
auto_trans_p5 = auto_trans_p5.rename(columns={"D5AR": "Jobs_auto","D5BR": "Jobs_transit_p5" })
auto_trans_p75 = auto_trans_p75.rename(columns={"D5AR": "Jobs_auto","D5BR": "Jobs_transit_p75" })

In [10]:
auto_trans_p5.to_csv("output/job_auto_and_transit_accessibity45mins_p5.csv")
auto_trans_p75.to_csv("output/job_auto_and_transit_accessibity45mins_p75.csv")